# <span style='font-family:rockwell'>  <span style='color:black'> ***Bradford Assay***
### <span style='font-family:rockwell'>  <span style='color:black'> **Data Processor**

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
import io 
from IPython.display import display, HTML, clear_output
import numpy as np
import scipy.stats as stats
import warnings
import base64

#Eliminate warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.simplefilter(action='ignore', category=FutureWarning)

out = widgets.Output()
display(out)

uploader = widgets.FileUpload(description = "Upload your file: xlsx/xls/csv", multiple=False, button_style = 'info')
uploader.layout.width = '300px'
display(uploader)


def on_upload_change(self):
    global content, file_title_csv
    with out:   
        for file_name in uploader.value:
            extension = file_name.split('.')[1]
            file_title_csv = file_name.split('.')[0] + ' - processed_data' + '.csv'
            content = uploader.value[file_name]['content']

        if(extension == 'xlsx' or extension == 'xls'):
            uploader.close()
            content = pd.read_excel(io.BytesIO(content))
            return display(Header_text), display(content), display(process_btn)
        
        elif(extension == 'csv'):
            uploader.close()
            content = pd.read_csv(io.BytesIO(content))
            return display(Header_text), display(content), display(process_btn)

        else:
            uploader._counter = 0
            display(widgets.HTML(
    value="<H4><b> File not accepted: Only xlsx/xls/csv </b> </H4>"))
    


uploader.observe(on_upload_change, names='value') 

Header_text = widgets.HTML(
    value="<H1><b> File Contents </b> </H1>"
)



In [ ]:
out_2 = widgets.Output()
display(out_2)

process_btn = widgets.Button(description = "Process data", button_style='success')
    
# calculates the concentration using the absorbance_value, m & c values  
def calconc (gradient, intercept, absorbance):
    concentration = (absorbance - intercept) / gradient
    return round(concentration, 3)
        
# calculates required volume to get the the desired amount of proteins from a sample under a particular condition
def calcvol (Protein_μg_aliquot, Aliquot_volume_μl, desired_protein_μg):
    required_volume = (desired_protein_μg*Aliquot_volume_μl)/Protein_μg_aliquot    
    return round(required_volume, 3)

# calculates the amount of proteins in the entire sample using aliquot amounts
def calcsampleconc (Protein_μg_aliquot, ALiquot_volume_μl, Sample_volume_ml):
    Sample_volume_μl = Sample_volume_ml*1000
    Protein_μg_sample = (Sample_volume_μl*Protein_μg_aliquot)/ALiquot_volume_μl
    return Protein_μg_sample

def data_processing(on_upload_change):
        global conc, abso, m, c, content_2
        with out_2:
            
            process_btn.close()
            display(Header1_text)
            
            content['Condition_name'] = content['Condition_name'].str.lower()
            content['Standard_Unknown'] = content['Standard_Unknown'].str.lower()
            
            #reading data from the columns
            conc = content[content.Standard_Unknown =='s']['Protein_μg_sample']
            abso = content[content.Standard_Unknown =='s']['Absorbance_nm']
            
            #line of best fit using polyfit function
            m, c = np.polyfit(conc, abso, 1)
                        
            #group by Condition num/name, avg the abso values, name the new columns, round the avg values
            content_mean = content.groupby(['Condition_number'])['Absorbance_nm'].mean().round(3).rename('Average_absorbance_nm').reset_index()

            #merge the new column with the main index
            content_1 = content.merge(content_mean)
            
            #calculate the amount of proteins using the absorbance values
            content_1.loc[content_1.Standard_Unknown =='u','Protein_μg_aliquot'] = calconc(m, c, content_1['Average_absorbance_nm'])
            
            #drop unnecessary columns
            content_2 = content_1.drop(['Absorbance_nm', 'Replicate_number'], 1)

            #drop repetetive rows
            content_2.drop_duplicates(['Condition_number', 'Condition_name'], keep='first', inplace=True)
            content_2.reset_index(drop=True, inplace=True)
            
            #calculate volume to get desired amount of proteins. Default set to 100μg
            content_2.loc[content_2.Standard_Unknown =='u', ['Volume_(μl)_for_100μg']] = calcvol (content_2['Protein_μg_aliquot'], content_2['Aliquot_volume_μl'], 100)
            #calculate the amount of protein in entire sample based on amounts in aliquot 
            content_2.loc[content_2.Standard_Unknown =='u','Protein_μg_sample'] = calcsampleconc(content_2['Protein_μg_aliquot'], content_2['Aliquot_volume_μl'], content_2['Sample_volume_ml'])
            
            return display(content_2), display(widgets.HBox([graph_btn, download_btn]))
            

process_btn.on_click(data_processing)
Header1_text = widgets.HTML(
    value="<H1><b> Processed Data </b> </H1>")

In [ ]:
out_3 = widgets.Output()
display(out_3)

graph_btn = widgets.Button(description = "Display graph", button_style='info')

def graph_creator(self):
        with out_3:
            #layout of the graph
            plt.ylabel('Absorbance at 595nm')
            plt.xlabel('Amount of proteins (μg)')
            plt.title('Graph of the standard curve')
            
            #plot the data points,   # plot the line of best fit
            plt.plot(conc, abso, 'o')
            plt.plot(conc, m*conc+c, 'g-')

            plt.legend(['Standards', 'Line of best fit'])
            plt.text(-1, .28, r"y = {}x + {}".format(round(m, 4), round(c, 4)), color="k", fontsize=10)
            
            res = stats.linregress(conc, abso)
            plt.text(-1, .24, f"R-squared: {res.rvalue**2:.6f}", color="k", fontsize=10)
            return plt.show()
        
graph_btn.on_click(graph_creator)

out_4 = widgets.Output()
display(out_4)

def trigger_download(text, filename, kind='text/json'):
    
    testt=content_2.to_csv(index = False)
    content_b64 = base64.b64encode(testt.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with out_4:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

download_btn = widgets.Button(description='Download processed data', button_style = 'warning')
download_btn.layout.width = '200px'


def download_file(e=None):
    trigger_download(content_2, file_title_csv, kind='text/plain')

download_btn.on_click(download_file)